# Sound thermometer - binary classification (hot/cold)

Classifying fluid's temperature using sound. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Basic imports
import src
import numpy as np
import matplotlib.pyplot as plt
import  seaborn as sns
import tensorflow as tf

# File-loading imports
import os
import glob

# Keras imports
from keras.layers import Input, Resizing, Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Metric imports
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

### Setup

Dataset consists of number of .wav files sorted into two categories: HOT and COLD. Dataset will be divided into three: training, validation and test in a ratio of 8:1:1

In [3]:
DATASET = '../data/binary'

In [4]:
labels = np.array(os.listdir(DATASET))
labels

array(['hot', 'cool'], dtype='<U4')

In [5]:
for label in labels:
    print(f"Number in {label}: {len(os.listdir(f'{DATASET}/{label}'))}")

Number in hot: 231
Number in cool: 183


In [7]:
file_paths = np.array(glob.glob(f'{DATASET}/*/*'))
np.random.shuffle(file_paths)

len(file_paths)

414

In [8]:
num_files = len(file_paths)

border_1 = int(num_files * 0.8)
border_2 = (num_files - border_1) // 2

train_files = file_paths[:border_1]
val_files = file_paths[border_1:-border_2]
test_files = file_paths[-border_2:]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

Training set size 331
Validation set size 42
Test set size 41


### Preprocessing

During preprocessing phase waveform data is transformed into spectrogram using Short Time Fourier's Transform. Data can also be augmented by adding white noise, shifting or stretching sound. Output of the generate_dataset method is tf.data.Dataset object.

In [12]:
train_generator = src.DataGenerator(train_files, labels, {src.add_white_noise: 0.005, src.shift_sound: 1600.0, src.stretch_sound: 1.0})
val_generator = src.DataGenerator(train_files, labels, {})
test_generator = src.DataGenerator(train_files, labels, {})

In [14]:
train_ds = train_generator.generate_dataset()
val_ds = val_generator.generate_dataset()
test_ds = test_generator.generate_dataset()

In [15]:
model = tf.keras.Sequential([
    Input(shape=(124, 129, 1)),
    # Down sample the input.
    Resizing(32, 32),

    Conv2D(16, kernel_size=(2, 2), padding='same',activation='relu'),
    MaxPooling2D(),

    Conv2D(32, kernel_size=(2, 2), padding='same', activation='relu'),
    MaxPooling2D(),

    Conv2D(64, kernel_size=(2, 2), padding='same', activation='relu'),
    MaxPooling2D(),

    Dropout(0.2),
    Flatten(),

    Dense(128),
    Dense(2, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 32, 32, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 16)        80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 64)          8